In [1]:
import pylab
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
from scipy.spatial import distance
from scipy.optimize import curve_fit
from scipy.optimize import minimize
from scipy.optimize import basinhopping,differential_evolution
from mpl_toolkits.axes_grid1 import make_axes_locatable
from scipy import stats


In [2]:
## Model structure
Mt = 40
Nt = 20

Ma = 20
Na = 3

## Model parameters

# Unisensory receptive fields
phit_0 = 1
sigmat_phi = 0.5
phia_0 = 1
sigmaa_phi = 10

# External stimuli
it_0 = 2.5
sigmat_i = 0.3
sigmat_v = 0.1
xt_0 = 10 
yt_0 = 5

ia_0 = 3.6
sigmaa_i = 0.3
sigmaa_v = 0.4
ya_0 = 5
xa_0 = 100

tt_0 = 0
ta_0 = 0

In [3]:
## Receptive fields

# Tactile RF centres
xt = np.arange(1,Mt+1)*0.5 
yt = np.arange(1,Nt+1)*0.5

# Auditory RF centres
xa = (np.arange(1,Ma+1)*10)-5
ya = (np.arange(1,Na+1)*10)-15
              
def phit(x,y):
    phi = np.zeros((Mt,Nt))
    for i in range(Mt):
        for j in range(Nt):
            phi[i][j] = phit_0*np.exp(-((np.square(xt[i]-x)+np.square(yt[j]-y))/(2*np.square(sigmat_phi))))
    return phi

def phia(x,y):
    phi = np.zeros((Ma,Na))
    for i in range(Ma):
        for j in range(Na):
            phi[i][j] = phia_0*np.exp(-((np.square(xa[i]-x)+np.square(ya[j]-y))/(2*np.square(sigmaa_phi))))
    return phi

# Calculation
dif = 0.2
xt_i = np.arange(0,20+dif,dif)
yt_n = np.arange(0,10+dif,dif)

phi_t = np.zeros((Mt,Nt,len(xt_i),len(yt_n)))        
for k in range(len(xt_i)):
    for l in range(len(yt_n)):
        phi_t[:,:,k,l] = phit(xt_i[k],yt_n[l])
        
xa_i = np.arange(0,200+dif,dif)
ya_n = np.arange(0,30+dif,dif)

phi_a = np.zeros((Ma,Na,len(xa_i),len(ya_n)))        
for k in range(len(xa_i)):
    for l in range(len(ya_n)):
        phi_a[:,:,k,l] = phia(xa_i[k],ya_n[l])
        

## External stimulus
def stimt(x,y,t):
    if t<tt_0: 
        I = 0
    else: 
        #v = np.random.normal(0, sigmat_v)
        #I = (it_0+sigmat_v*v)*np.exp(- (np.square(xt_0-x)+np.square(yt_0-y))/(2*np.square(sigmat_i)))
        I = (it_0)*np.exp(- (np.square(xt_0-x)+np.square(yt_0-y))/(2*np.square(sigmat_i)))
    return I 

def stima(x,y,t,xa_0):
    if t<ta_0: 
        I = 0
    else: 
        #v = np.random.normal(0, sigmaa_v)
        #I = (ia_0+sigmaa_v*v)*np.exp(- (np.square(xa_0-x)+np.square(ya_0-y))/(2*np.square(sigmaa_i)))
        I = (ia_0)*np.exp(- (np.square(xa_0-x)+np.square(ya_0-y))/(2*np.square(sigmaa_i)))
    return I 

## Unisensory input
def PHIt(t):

        PHI = np.zeros((Mt,Nt,len(xt_i),len(yt_n)))        
        
        for k in range(len(xt_i)):
            for l in range(len(yt_n)):
                PHI[:,:,k,l] = np.multiply(phi_t[:,:,k,l],stimt(xt_i[k],yt_n[l],t))
        PHI = np.sum(PHI,axis=3)
        PHI = np.sum(PHI,axis=2)
        
        return PHI
    
def PHIa(t,xa_0):

        PHI = np.zeros((Ma,Na,len(xa_i),len(ya_n)))        
        
        for k in range(len(xa_i)):
            for l in range(len(ya_n)):
                PHI[:,:,k,l] = np.multiply(phi_a[:,:,k,l],stima(xa_i[k],ya_n[l],t,xa_0))
        PHI = np.sum(PHI,axis=3)
        PHI = np.sum(PHI,axis=2)
        
        return PHI

In [4]:
## Lateral connections in unisensory areas

def Lw(Lt_ex,Lt_in,sigmat_ex,sigmat_in,La_ex,La_in,sigmaa_ex,sigmaa_in):
    # Tactile Connections
    Lt = np.zeros((Mt*Nt,Mt*Nt))

    for i in range(Mt*Nt):
        for j in range(Mt*Nt):
            if i==j: 
                Lt[i,j] = 0
            else:
                Dtx = xt[np.floor_divide(i,Nt)] - xt[np.floor_divide(j,Nt)]
                Dty = yt[np.remainder(i,Nt)] - yt[np.remainder(j,Nt)]
                Lt[i,j] = Lt_ex*np.exp(- (np.square(Dtx)+np.square(Dty))/(2*np.square(sigmat_ex)))-Lt_in*np.exp(- (np.square(Dtx)+np.square(Dty))/(2*np.square(sigmat_in)))

    # Auditory Connections
    La = np.zeros((Ma*Na,Ma*Na))

    for i in range(Ma*Na):
        for j in range(Ma*Na):
            if i==j: 
                La[i,j] = 0
            else:
                Dax = xa[np.floor_divide(i,Na)] - xa[np.floor_divide(j,Na)]
                Day = ya[np.remainder(i,Na)] - ya[np.remainder(j,Na)] #before was remainder-1
                La[i,j] = La_ex*np.exp(- (np.square(Dax)+np.square(Day))/(2*np.square(sigmaa_ex)))-La_in*np.exp(- (np.square(Dax)+np.square(Day))/(2*np.square(sigmaa_in)))
    return Lt,La            
                
# Lateral inputs
def LIt(z):
    # z is a matrix of MtxNt dimensions
    LI = np.zeros(Mt*Nt)
    z = np.reshape(z,(1,Mt*Nt))
    for i in range(Mt*Nt):
            LI[i] = np.sum(np.multiply(Lt[i,:],z[0,:])) 
    LI = np.reshape(LI,(Mt,Nt))
    return LI

def LIa(z):
    # z is a matrix of MtxNt dimensions
    LI = np.zeros(Ma*Na)
    z = np.reshape(z,(1,Ma*Na))
    for i in range(Ma*Na):
            LI[i] = np.sum(np.multiply(La[i,:],z[0,:])) 
    LI = np.reshape(LI,(Ma,Na))
    return LI

In [274]:
## Feedforward and feedback synapses

def Fw (Wt_0,Wa_0,Bt_0,Ba_0):
    k1 = 15.41902756
    k2 = 813.75069556
    alpha = 0.9 

    # Tactile connections
    Bt = np.ones((Mt,Nt))*Bt_0
    Wt = np.ones((Mt,Nt))*Wt_0

    # Auditory connections
    Ba = np.zeros((Ma,Na))
    Wa = np.zeros((Ma,Na))

    lim = 66.0879161

    for i in range(Ma):
        for j in range(Na):
            if (xa[i]<lim) & (ya[j]<20):
                D = 0
            else: 
                D = distance.euclidean((xa[i],ya[j]),(lim,ya[j]))              
            Ba[i,j] = alpha*Ba_0*np.exp(- D/k1)+(1-alpha)*Ba_0*np.exp(- D/k2)
            Wa[i,j] = alpha*Wa_0*np.exp(- D/k1)+(1-alpha)*Wa_0*np.exp(- D/k2)
    return Wt,Wa,Bt,Ba

# Feedback inputs
def bt(z):
    #bt = np.zeros(Mt,Nt)
    bt = np.multiply(Bt,z)
    return bt

def ba(z,Ba):
    #bt = np.zeros(Mt,Nt)
    ba = np.multiply(Ba,z)
    return ba


In [6]:
## Neuron activity in unisensory areas
ft_min = -0.12
ft_max = 1
qt_c = 19.43
rt = 0.34

fa_min = -0.12
fa_max = 1
qa_c = 19.43
ra = 0.34

tau = 20

def psit(qt,b):
    y = qt
    for i in range(Mt):
        for j in range(Nt):
            y[i,j] = (ft_min+ft_max*np.exp((qt[i,j]-qt_c)*rt+b))/(1+np.exp((qt[i,j]-qt_c)*rt+b))
    return y

def psia(qa,b):
    y = qa
    for i in range(Ma):
        for j in range(Na):
            y[i,j] = (fa_min+fa_max*np.exp((qa[i,j]-qa_c)*ra+b))/(1+np.exp((qa[i,j]-qa_c)*ra+b))
    return y

## Multisensory neuron activity
fm_min = 0
fm_max = 1
qm_c = 12
rm = 0.6

def psim(qm):
    y = (fm_min+fm_max*np.exp((qm-qm_c)*rm))/(1+np.exp((qm-qm_c)*rm))
    return y

## Evolutionary prunning mechanism
def prun(WM,pr):
    newM = np.copy(WM)
    newM[newM < pr] = 0
    return newM

In [275]:
## Experiment function - Canzonieri 2012 experiment

def experimentrun(a_distances,time,b,pr):
    dt = 0.4
    timesteps = int(time/dt)
    ndist = len(a_distances)
    
    RTs = np.zeros(ndist)
    ti = PHIt(0) 
    PrBa = prun(Ba,pr)
    PrWa = prun(Wa,pr*2.6)
    dtau = dt/tau
    
    qt = np.zeros((Mt,Nt,timesteps+1,ndist))
    ut = np.zeros((Mt,Nt,timesteps+1,ndist))
    zt = np.zeros((Mt,Nt,timesteps+1,ndist))
    pt = np.zeros((Mt,Nt,timesteps+1,ndist))

    qa = np.zeros((Ma,Na,timesteps+1,ndist))
    ua = np.zeros((Ma,Na,timesteps+1,ndist))
    za = np.zeros((Ma,Na,timesteps+1,ndist))
    pa = np.zeros((Ma,Na,timesteps+1,ndist))

    qm = np.zeros((timesteps+1,ndist))
    um = np.zeros((timesteps+1,ndist))
    zm = np.zeros((timesteps+1,ndist))
    pm = np.zeros((timesteps+1,ndist))
    
    ZT = np.zeros((1,timesteps+1,ndist))
    
    for d in range(ndist):
        xa_0 = a_distances[d]     
        ai = PHIa(0,xa_0)

        for i in range(timesteps):    
            # Tactile activity
            ut[:,:,i+1,d] = ti+LIt(zt[:,:,i,d])+bt(zm[i,d])
            qt[:,:,i+1,d] = qt[:,:,i,d] + dtau*(-qt[:,:,i,d]+ut[:,:,i,d])
            pt[:,:,i+1,d] = psit(qt[:,:,i,d],b)
            zt[:,:,i+1,d] = pt[:,:,i,d]*np.heaviside(pt[:,:,i,d],0)

            ZT[0,i+1,d] = np.sum(zt[:,:,i,d])

            # Auditory activity
            ua[:,:,i+1,d] = ai+LIa(za[:,:,i,d])+ba(zm[i,d],PrBa) 
            qa[:,:,i+1,d] = qa[:,:,i,d] + dtau*(-qa[:,:,i,d]+ua[:,:,i,d])
            pa[:,:,i+1,d] = psia(qa[:,:,i,d],b)
            za[:,:,i+1,d] = pa[:,:,i,d]*np.heaviside(pa[:,:,i,d],0)

            # Multisensory activity
            um[i+1,d] = np.sum(np.multiply(Wt,zt[:,:,i,d])) + np.sum(np.multiply(PrWa,za[:,:,i,d]))
            qm[i+1,d] = qm[i,d] + dtau*(-qm[i,d]+um[i,d])
            pm[i+1,d] = psim(qm[i,d])
            zm[i+1,d] = pm[i,d]*np.heaviside(pm[i,d],0)    

        RT = dt*(np.abs(ZT[:,:,d] - ZT[0,timesteps,d]*0.9)).argmin()
        RTs[d] = RT
    return RTs

## Sigmoid function to fit
def RTsig(d,etamin,etamax,dc,h):
    return (etamin+etamax*np.exp((d-dc)/h))/(1+np.exp((d-dc)/h))

## Sigmoid function fitting
def sigfit(RTs,a_distances):
    t1 = np.linspace(38, 112, 100) # These boundaries may change if the velocity is different from 30 cm/s
    popt, pcov = curve_fit(RTsig, a_distances, RTs,bounds=([0,0,38,0],[100,200,112,100])) # Same as above. Beware of boundaries.
    sigpar = np.asarray(popt)

    etamin = sigpar[0] 
    etamax = sigpar[1]
    dc = sigpar[2]
    h = sigpar[3]
    return etamin,etamax,dc,h

In [8]:
HCdata = pd.read_excel (r'C:\\Users\\renat\Dissertation\PPS_data_SCZ_SPQ.xlsx',sheet_name='HC') #for an earlier version of Excel, you may need to use the file extension of 'xls'
SCZdata = pd.read_excel (r'C:\\Users\\renat\Dissertation\PPS_data_SCZ_SPQ.xlsx',sheet_name='SCZ') #for an earlier version of Excel, you may need to use the file extension of 'xls'
SPQdata = pd.read_excel (r'C:\\Users\\renat\Dissertation\PPS_data_SCZ_SPQ.xlsx',sheet_name='high spq') #for an earlier version of Excel, you may need to use the file extension of 'xls'

sets = [HCdata,SCZdata,SPQdata]
RTdata = []

for i in sets:
    subjects = i['Subject'].unique()
    delays = i[(i['Sound']=='Loom')&(i['Cond'] != 'Loom_NO')]['Cond'].unique()
    delays = np.delete(delays,np.where(delays=='Loom_-700')[0])
    idx = [3,4,0,1,2]
    delays = delays[idx]
    RTs = []

    for s in subjects:
        meanRTs = []
        for t in delays:
            RT = i[(i['Subject'] == s)&(i['Cond'] == t)]['Rtpul']           
            meanRTs.append(np.mean(RT))
        RTs.append(meanRTs)
    RTdata.append(RTs)
    
HC_RT = np.asarray(RTdata[0])
SCZ_RT = np.asarray(RTdata[1])
SPQ_RT = np.asarray(RTdata[2])

In [276]:
## Experiments setup. 

# Calculate distance in cms. Speaker is positioned at 100cm from the hand. 
times = np.asarray([300,800,1500,2200,2700])
timesweep = np.arange(300,2800,100)
ts = timesweep/1000
v = 30 #22 cm/s / 28 gives good fit as well / 30.0125 best

# Initial setup
trials = 1 #18 in behavioural study
a_distances = (120 - ts*v) # distance points in cm (network coordinates)
simtime = 200 #ms # 0.1 in behavioural study? 
# Synapses
Lt,La = Lw(0.15,0.05,1,4,0.15,0.05,20,80)
Wt,Wa,Bt,Ba = Fw(6.5,6.5,2.5,2.5) 

In [268]:
idx = [0,5,12,19,24]
distances = np.take(a_distances,idx)
edata = np.mean(HC_RT,axis=0)

def hcmodelrun(theta): 
    hcRTs = experimentrun(trials,distances,simtime,theta[0],theta[1])
    yf = edata 
    xf = hcRTs
    
    m = (xf.size * np.sum(xf*yf) - np.sum(xf) * np.sum(yf)) / (xf.size*np.sum(xf*xf) - np.sum(xf) ** 2)
    bias = (np.sum(yf) - m*np.sum(xf)) / xf.size
    
    cost = np.sum(np.square(np.divide(yf-(m*xf+bias),yf)))
    
    return cost*100000
    
bounds=[(-1.25,4.5),(0,2.5)]
res = differential_evolution(hcmodelrun, bounds)

In [ ]:
print(res)

In [79]:
idx = [0,5,12,19,24]
distances = np.take(a_distances,idx)
edata = np.mean(SCZ_RT,axis=0)

def sczmodelrun(theta):
    sczRTs,ZMs,ZTs,ZAs = experimentrun(trials,distances,simtime,theta[0],theta[1])
    yf = edata
    xf = sczRTs[0]
    
    m = (xf.size * np.sum(xf*yf) - np.sum(xf) * np.sum(yf)) / (xf.size*np.sum(xf*xf) - np.sum(xf) ** 2)
    bias = (np.sum(yf) - m*np.sum(xf)) / xf.size
    
    cost = np.sum(np.square(np.divide(yf-(m*xf+bias),yf)))
    
    return cost*100000

bounds=[(-1.25,4.5),(0,2.5)]
res = differential_evolution(sczmodelrun, bounds)

In [ ]:
print(res)

In [243]:
idx = [0,5,12,19,24]
distances = np.take(a_distances,idx)
edata = np.mean(SPQ_RT,axis=0)

def spqmodelrun(theta):
    spqRTs,ZMs,ZTs,ZAs = experimentrun(trials,distances,simtime,theta[0],theta[1])
    yf = edata
    xf = spqRTs[0]
    
    m = (xf.size * np.sum(xf*yf) - np.sum(xf) * np.sum(yf)) / (xf.size*np.sum(xf*xf) - np.sum(xf) ** 2)
    bias = (np.sum(yf) - m*np.sum(xf)) / xf.size
    
    cost = np.sum(np.square(np.divide(yf-(m*xf+bias),yf)))
    
    return cost*10000

bounds=[(-1.25,4.5),(0,2.5)]
res = differential_evolution(spqmodelrun, bounds)

In [ ]:
print(res)
